# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [3]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [5]:
## your code to subset to one press release and take the string
subset = doj[doj["id"]=="17-1204"]
pharma = subset["contents"].iloc[0]
pharma

## to check that it is a string:
#type(pharma)

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [7]:
## Part A:
pharma_processed = [token for token in wordpunct_tokenize(pharma)
                   if token.isalpha()]
#pharma_processed

In [9]:
## Part B:
pharma_pos = pos_tag(pharma_processed)

#pharma_pos

In [11]:
## Part C:
adj_tags = ["JJ", "JJR", "JJS"]
all_adjectives = [tok[0] for tok in pharma_pos
                 if tok[1] in adj_tags]

adj_series = pd.Series(all_adjectives)
adj_counts = adj_series.value_counts()

top_5_adj = adj_counts.head(5).reset_index()
top_5_adj.columns = ["adjective", "count"]
top_5_adj

,adjective,count
0,former,8
1,opioid,5
2,nationwide,4
3,addictive,3
4,other,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [13]:
## Part A:
spacy_pharma = nlp(pharma)
#print(type(spacy_pharma))

In [15]:
## Part B:
law_entities = []
for tok in spacy_pharma.ents:
    if tok.label_ == "LAW" and tok.text not in law_entities: 
        law_entities.append(tok.text)

print(law_entities)

['RICO', 'the Controlled Substances Act']


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

**Part C:**

According to Herman Law White Collar Criminal Defense, a violation of RICO occurs when a person, in connection with an enterprise, engages in a pattern of racketeering activity that can include the distribution of a controlled substance or wire fruad, meaning that this pharmaceutical kickbacks case can be classified under RICO because it relates to the distribution of opiods (a controlled substance) through a company (Insys Therapeutics) in a way that violates anti-kickback law and involves wire fraud.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [27]:
## Part D:
sentence_lengths =[]
for tok in spacy_pharma.ents:
    if tok.label_ == "DATE" and re.search(r"years?", tok.text.lower()) and tok.text not in sentence_lengths:
        sentence_lengths.append(tok.text)
        
print(sentence_lengths)

['last year', '20 years', 'three years', 'five years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [29]:
## PART E:
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(pharma) #breaks pharma into sentences
year_sentences = [s for s in sentences if re.search(r"years?", s.lower())]

print("Sentences that mention year(s):\n")
for s in year_sentences:
    print("-", s, "\n")

Sentences that mention year(s):

- "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. 

- This investigation highlights our commitment to defending our mail system from illegal misuse and ensuring public trust in the mail.”“The U.S. Department of Veterans Affairs, Office of Inspector General will continue to aggressively investigate those that attempt to fraudulently impact programs designed to benefit our veterans and their families,” said Donna L. Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss. 

- The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised releas

**Part E:**

If convicted after this indictment, the CEO may be facing up to 20 years in prison and up to 3 years of probation each for his RICO charge and his mail/wire fraud charge; he may face up to 5 years in prison and up to 3 years of probation for his violation of Anti-Kickback Law charge.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [31]:
## PART A:
doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]
doj_subset.shape

(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [33]:
## your code here to define function
def func(string):
    ## Remove named entities from each press release:
    spacy_string = nlp(string)
    named_entities = [tok.text for tok in spacy_string.ents]
    if named_entities:
        pattern = "(" + "|".join([re.escape(ent) for ent in named_entities]) + ")" #this groups the named entities from the string together so that we can get rid of them in the next line
        # we need to use re.escape so that our regex pattern isn't thrown off by the presence of special characters in the strings
        cleaned_text = re.sub(pattern, "", string) #any time one of the named entities (given by the pattern) appears in the string, it will be replaced with ""

## I think it would also be possible to do this step via a for loop that loops over each entity in the named_entities
## list and replaces it with "" in the string, but I'm pretty sure it would take a lot longer since you'd have to loop
## through the string one time for each named entity rather than looping through it once and checking for any named entity
    
    else:
        cleaned_text = string
    #just a check:
    #print(cleaned_text)
    
    ## Score the sentiment of the entire press release using SentimentIntensityAnalyzer and polarity_scores
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(cleaned_text)

    ## Returns the length-four sentiment dictionary in any order
    return sentiment_scores

In [35]:
## Running it on a few examples:
func("Barack Obama spoke at the United Nations in 2014. He said the U.S. economy had improved.")
func("The U.S. News and Report (a special website) named Dartmouth College as one of the top universities. This made all of its undergraduates extremely happy, so they celebrated. People who don't go to Dartmouth College were very angry about this and called them up.")


{'neg': 0.0, 'neu': 0.744, 'pos': 0.256, 'compound': 0.4767}

{'neg': 0.078, 'neu': 0.651, 'pos': 0.271, 'compound': 0.835}

In [37]:
## your code here executing the function:
doj_sentiment_scores = [func(press_rel) for press_rel in doj_subset["contents"]]

#check:
#doj_sentiment_scores

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [45]:
## PART C:
doj_subset_wscore = doj_subset.copy()
doj_subset_wscore["sentiment_scores"] = doj_sentiment_scores
doj_subset_wscore["neg"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["neg"])
doj_subset_wscore["neu"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["neu"])
doj_subset_wscore["pos"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["pos"])
doj_subset_wscore["compound"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["compound"])

highest_neg_scores = doj_subset_wscore.sort_values(by="neg", ascending=False)
print(highest_neg_scores[["id", "contents", "neg"]].head(2))

# also without print, since the formatting is kinda ugly:
highest_neg_scores[["id", "contents", "neg"]].head(2)

         id  \
329  14-248   
572  13-312   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.323
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.303


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [47]:
## Part D:
mean_sentiment_by_topic = doj_subset_wscore.groupby("topics_clean")["compound"].agg("mean")
mean_sentiment_by_topic


topics_clean
Civil Rights             -0.093931
Hate Crimes              -0.930943
Project Safe Childhood   -0.681391
Name: compound, dtype: float64

## Part E:
We likely observe this variation in scores because different emotional tones and public perceptions are associated with each issue; for instance, Hate Crimes and Project Safe Childhood may have more negative sentiments due to their often tragic nature while Civil Rights may be viewed more neutrally, considering there are positive contexts in which Civil Rights-related words are discussed.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [49]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [51]:
## your code defining a text processing function
## PART A:
def preprocess(string):
    
    ## Convert word to lowercase:
    str_lower = string.lower()

    ## Remove stopwords, adding custom stopwords to default list:
    stopword_list = stopwords.words("english")
    new_stopwords_list = stopword_list + custom_doj_stopwords
    str_token = [word for word in wordpunct_tokenize(str_lower) if word not in new_stopwords_list]

    ## Only retains alpha words and words with more than 3 characters using the snowball stemmer from nltk to stem:
    stemmer = SnowballStemmer("english")
    str_stem = [stemmer.stem(tok) for tok in str_token if tok.isalpha() and len(tok) > 3]

    ## Return a joined preprocessed string:
    return " ".join(str_stem)


In [53]:
## your code executing the function
## PART B:
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(preprocess)

#check:
# doj_subset_wscore[["contents", "processed_text"]]

In [55]:
## your code showing the examples
## PART C:
id_list = ["16-718", "16-217"]
example = doj_subset_wscore[doj_subset_wscore["id"].isin(id_list)][["id", "contents", "processed_text"]]
example

,id,contents,processed_text
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr consti

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [57]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [59]:
# PART A
strings_list = doj_subset_wscore["processed_text"]
updated_doj = doj_subset_wscore.rename(columns={"compound": "compound_sent"})
#to avoid duplicated "compound" or "id" columns once we create the dtm

data = updated_doj[["id", "compound_sent", "topics_clean"]]

dtm = create_dtm(strings_list, data)
dtm

,index,id,compound_sent,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zane,zealand,zealous,zeeman,zero,zionism,zobel,zone,zunggeemog,zwengel
0,77,17-1235,-0.9931,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,155,15-1522,-0.9325,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,157,16-213,-0.7579,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,162,16-381,-0.9037,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,168,14-464,-0.9864,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,13002,09-368,-0.9689,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,13032,18-775,0.7003,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,13034,12-596,-0.9648,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,13068,18-359,-0.9798,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
## Defining the function:

def get_topwords(df_sub):
    word_columns = df_sub.drop(columns=["index", "id", "compound_sent", "topics_clean"])
    word_sums = word_columns.sum().sort_values(ascending=False)
    print(word_sums.head(10))



    

In [63]:
## PART B:
cutoff = dtm["compound_sent"].quantile(0.95)
b_sub = dtm[dtm["compound_sent"] > cutoff]

print("Top 10 words for press releases with compound sentiment in the top 5%:")
get_topwords(b_sub)

## PART C:
cutoff2 = dtm["compound_sent"].quantile(0.05)
c_sub = dtm[dtm["compound_sent"] < cutoff2]

print("\nTop 10 words for press releases with compound sentiment in the bottom 5%:")
get_topwords(c_sub)

## PART D:
d_sub_cr = dtm[dtm["topics_clean"]=="Civil Rights"]
print("\nTop 10 words for Civil Rights press releases:")
get_topwords(d_sub_cr)

d_sub_hc = dtm[dtm["topics_clean"]=="Hate Crimes"]
print("\nTop 10 words for Hate Crimes press releases:")
get_topwords(d_sub_hc)

d_sub_safe = dtm[dtm["topics_clean"]=="Project Safe Childhood"]
print("\nTop 10 words for Project Safe Childhood press releases:")
get_topwords(d_sub_safe)


Top 10 words for press releases with compound sentiment in the top 5%:
agreement     171
enforc        130
state         110
disabl        107
ensur         107
communiti      99
student        87
settlement     86
servic         83
general        82
dtype: int64

Top 10 words for press releases with compound sentiment in the bottom 5%:
assault     191
crime       171
victim      164
hate        129
defend      126
offic       119
conspir     105
sentenc     104
charg        97
american     95
dtype: int64

Top 10 words for Civil Rights press releases:
offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64

Top 10 words for Hate Crimes press releases:
victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

Top 10 words for Project Safe Childhoo

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [65]:
# PART A:
#Step 1: re-tokenize our preprocessed text
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  doj_subset_wscore.processed_text]

#Step 2: use genism create dictionary
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict)

#Step 3: Skip filtering out rare and common words

#Step 4: apply dictionary to tokenized text
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

#Step 5: estimate the model
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict, 
                                         passes=15, 
                                         random_state=100,
                                         alpha = 'auto',
                                         per_word_topics = True)

#check
#print(type(ldamod))

In [66]:
# PART B:
topics = ldamod.print_topics(num_words = 15)
## printed in this format so it would be easier to compare the list of words in each topic
## with our earlier exercise using pre-determined topics
for topic_num, topic_words in topics:
    print(f"Topic {topic_num + 1}: ")
    print(" -" + topic_words.replace(" + ", "\n -").replace("*", ": "))
    print("\n")


Topic 1: 
 -0.019: "child"
 -0.013: "exploit"
 -0.011: "sexual"
 -0.010: "sentenc"
 -0.010: "prosecut"
 -0.010: "crimin"
 -0.009: "offic"
 -0.009: "victim"
 -0.009: "safe"
 -0.009: "childhood"
 -0.008: "project"
 -0.008: "pornographi"
 -0.008: "year"
 -0.008: "children"
 -0.008: "investig"


Topic 2: 
 -0.011: "hous"
 -0.011: "discrimin"
 -0.009: "disabl"
 -0.008: "enforc"
 -0.007: "agreement"
 -0.007: "state"
 -0.007: "said"
 -0.006: "court"
 -0.006: "violat"
 -0.006: "alleg"
 -0.006: "feder"
 -0.005: "requir"
 -0.005: "general"
 -0.005: "settlement"
 -0.005: "fair"


Topic 3: 
 -0.011: "victim"
 -0.011: "crime"
 -0.011: "defend"
 -0.010: "charg"
 -0.010: "hate"
 -0.009: "prosecut"
 -0.009: "said"
 -0.009: "sentenc"
 -0.008: "american"
 -0.008: "feder"
 -0.008: "guilti"
 -0.008: "indict"
 -0.008: "assault"
 -0.007: "african"
 -0.007: "year"




## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [69]:
## PART A:
topic_probs = [ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]

# We want this to return true, which it does:
len(topic_probs) == len(doj_subset_wscore)


True

In [71]:
## PART B:
# Convert topic_probs into a DataFrame
topic_df = pd.DataFrame([[prob for i, prob in doc] for doc in topic_probs],
                        columns=["Topic_1", "Topic_2", "Topic_3"])

# Join topic probabilities to main DataFrame
doj_subset_wscore = doj_subset_wscore.reset_index(drop=True).join(topic_df)

# Add column for most probable topic
doj_subset_wscore["top_topic"] = topic_df.idxmax(axis=1)
#doj_subset_wscore

In [73]:
## PART C
topic_distribution = pd.crosstab(index = doj_subset_wscore["topics_clean"], columns = doj_subset_wscore["top_topic"], normalize = "index")

# Multiply everything by 100 to get percentages
topic_distribution = topic_distribution.mul(100)

print(topic_distribution)


top_topic                  Topic_1    Topic_2    Topic_3
topics_clean                                            
Civil Rights             10.163934  69.508197  20.327869
Hate Crimes               9.349593   0.000000  90.650407
Project Safe Childhood  100.000000   0.000000   0.000000


In [75]:
## PART D
# To look at examples:
doj_subset_wscore.sample(5)

## We believe that the Project Safe Childhood topic maps more cleanly onto our estimated topics because, comparing the list of top words
## for Project Safe Childhood press releases, we see that they are more distinct from the other two topic categories. Civil Rights and 
## Hate Crimes have more overlapping top words between them than either of them do with Project Safe Childhood words, so the LDA model 
## is not able to determine which topic Hate Crime and Civil Rights releases belong to as easily.

id  \
644  11-599   
329  16-588   
374  17-296   
111  09-081   
674  17-931   

                                                                                                                     title  \
644                                              Two Arkansas Men Plead Guilty to Firebombing an Interracial Couple’s Home   
329  Justice Department Reaches Extension Agreement to Improve Georgia’s Developmental Disability and Mental Health System   
374               Justice Department Settles Immigration-Related Discrimination Claim Against Florida Pizza Delivery Chain   
111                                                                   Final Defendant Pleads Guilty to Anti-Obama Assaults   
674                     Two Texas Men Plead Guilty to Federal Hate Crime for Assaults Based on Victim’s Sexual Orientation   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [77]:
## PART A:
def create_bigram_onedoc(proc_text_string):
    words = proc_text_string.split()
    bigrams = [f"{words[i]}_{words[i+1]}" for i in range(len(words)-1)]
    return " ".join(bigrams) #to return it as a string that is the paur of words with the birgrams structured with underscores

doj_subset_wscore["processed_text_bigrams"] = doj_subset_wscore["processed_text"].apply(create_bigram_onedoc)

# check:
#doj_subset_wscore.head()

In [79]:
## PART B:
example = doj_subset_wscore[doj_subset_wscore["id"]=="16-217"]
example[["id", "processed_text", "processed_text_bigrams"]]

,id,processed_text,processed_text_bigrams
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss vote settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quick quick_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa oro

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [81]:
## PART C:
strings_list_bigrams = doj_subset_wscore["processed_text_bigrams"]

updated_doj2 = doj_subset_wscore.rename(columns={"compound": "compound_sent"})
#to avoid duplicated "compound" or "id" columns once we create the dtm
data_bigram = updated_doj2[["id", "topics_clean", "compound_sent"]]

dtm_bigram = create_dtm(strings_list_bigrams, data_bigram)

#check:
#dtm_bigram

In [83]:
dtm.shape
dtm_bigram.shape

(717, 6870)

(717, 72721)

**Part D:** The bigram matrix has more dimensions, specifically columns, than the unigram matrix because when bigrams are made they include each word twice, once with the word before and once with the word after. This creates a large number of pairings. In a DTM, columns represent terms, and using bigrams inherently produces more terms.

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [85]:
# your code here


print("Civil Rights: ")
get_topwords(dtm_bigram[dtm_bigram["topics_clean"] == "Civil Rights"])
print("\n")

print("Hate Crimes: ")
get_topwords(dtm_bigram[dtm_bigram["topics_clean"] == "Hate Crimes"])
print("\n")

print("Project Safe Childhood: ")
get_topwords(dtm_bigram[dtm_bigram["topics_clean"] == "Project Safe Childhood"])
print("\n")

Civil Rights: 
fair_hous         231
deputi_general    221
princip_deputi    221
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
unit_state        156
nation_origin     143
consent_decre     128
dtype: int64


Hate Crimes: 
hate_crime          379
african_american    367
plead_guilti        275
year_prison         161
special_agent       124
racial_motiv        114
thoma_perez         111
grand_juri          101
perez_general        95
said_thoma           91
dtype: int64


Project Safe Childhood: 
safe_childhood       474
project_safe         472
child_pornographi    450
child_exploit        281
sexual_exploit       223
exploit_children     200
plead_guilti         197
exploit_obscen       176
obscen_section       175
child_sexual         174
dtype: int64




# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [66]:
## checking the og data:
#doj.head()

#checking the press releases that we want to work with (from question 1):
#doj_subset

In [60]:
## indictment-->charging someone wiht a crime
## conviction-->conviction after settlement/trial 
## sentencing-->x years of prison/supervised release post conviction

## PAIRS OF PRESS RELEASES where one is from one stage and another is from a different stage

## One of google's suggestions for finding pairwise similarity is TF-IDF & cosine similarity which measures overalp in word importance between documents
## This also stems from teh sklearn package which we discussed in class

## https://goodboychan.github.io/python/datacamp/natural_language_processing/2020/07/17/04-TF-IDF-and-similarity-scores.html 

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Get cleaned (but not overly processed) text from our data frame
texts = doj_subset["contents"] # google suggests notusing our overly processed text because we want entities and tfidf vectorizer will tokenize for us 

## TF-IDF vectorization 
## (term frequency-inverse document frequency = converts text into # vectors representing the importance of words within a document)
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)

## Computing cosine similarity (measure similarity between 2 vectors)
cosine_sim = cosine_similarity(tfidf_matrix)

## Remove self comparisons
np.fill_diagonal(cosine_sim, 0)


## need only unique pairs sicne cosine simmilarity is symmetirc (so avoid AB then BA)
pairs = []
num_doc = cosine_sim.shape[0]#gives the number of documents in your data set

## indexing into the cosine similarity matrix
for i in range(num_doc):
    for j in range(i+1, num_doc):
        pairs.append((cosine_sim[i,j], i, j))


top_pairs = sorted(pairs, reverse=True)[:2]

for sim_score, idx1, idx2 in top_pairs:
    
    print(f"Similarity Score: {cosine_sim[idx1, idx2]:.3f}\n")
    print(f"Index 1: {idx1}")
    print(f"Index 2: {idx2}")
    print("\n")

    doc1 = texts.iloc[idx1]
    doc2 = texts.iloc[idx2]

    df = pd.DataFrame({
        f"Document {idx1}": [doc1],
        f"Document {idx2}": [doc2]
    })

    display(df) #so you can read them side by side


## major difference seems to be pleading guilty vs being sentenced-->similar details just missing the year

Similarity Score: 0.986

Index 1: 203
Index 2: 686




,Document 203,Document 686
0,"A Church Hill, Maryland, resident was sentenced today to 20 years in prison to be followed by a lifetime term of supervised release for enticement of a minor to engage in sexual activity and attempting to transfer obscene materials to a minor, announced Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division and Acting U.S. Attorney Benjamin G. Greenberg of the Southern District of Florida. Lee Robert Moore, 38, pleaded guilty March 1, 2017, before U.S. District Judge Daniel T. K. Hurley of the Southern District of Florida. Moore was employed by the U.S. Secret Service-Uniformed Division and was assigned to the White House at the time of his arrest on Nov. 9, 2015, and has remained in custody since that time. Moore has since been terminated from his Secret Service position. According to admissions made in connection with his plea, Moore maintained a profile on the social media application “Meet24,” which provides a mobile-based platform for exchanging digital images, as well as voice and text messages. Delaware State Police Detectives with the Delaware Child Predator Task Force created a profile on this site, posing as a 14-year-old girl, with whom Moore engaged in a number of online chat sessions, via the “Meet24” and “Kik” mobile apps over a two-month period, including while Moore was at work. A number of the online chats between Moore and the undercover officers posing as a female minor were sexual in nature and, on several occasions, Moore sent pictures of himself, including one sexually explicit image. According to the plea documents, after his arrest, law enforcement discovered that Moore had communicated with a minor in Florida. Moore admitted that in those communications, he sent sexually explicit images of himself and enticed the minor to send sexually explicit photos of herself as well. Moore engaged in the same type of behavior with a 14-year-old girl in Texas and another 17-year-old girl in Missouri. Moore requested that his federal charges in Delaware be transferred to the Southern District of Florida so that he could plead guilty to both charges at one time. U.S. Immigration and Customs Enforcement’s Homeland Security Investigations and the Delaware Child Predator Task Force investigated the case. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Corey Steinberg of the Southern District of Florida prosecuted the case, with assistance from the U.S. Attorney’s Office for the District of Delaware. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ Offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit http://www.justice.gov/psc.","A Church Hill, Maryland resident pleaded guilty today in federal court to one count of enticement of a minor to engage in sexual activity and one count of attempting to transfer obscene materials to a minor, announced Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. Lee Robert Moore, 38, pleaded guilty today before U.S. District Judge Daniel T. K. Hurley of the Southern District of Florida. Moore was employed by the U.S. Secret Service-Uniformed Division and was assigned to the White House at the time of his arrest on Nov. 9, 2015, and has remained in custody since that time. Moore has since been terminated from his Secret Service position. According to admissions made in connection with his plea, Moore maintained a pro

Similarity Score: 0.958

Index 1: 462
Index 2: 463




,Document 462,Document 463
0,"A Weed, California man pleaded guilty today to conspiracy to produce child pornography for his participation in a website that was operated for the purpose of coercing and enticing minors as young as eight years old to engage in sexually explicit conduct on web camera. Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division; U.S. Attorney Dana J. Boente of the Eastern District of Virginia; and Assistant Director Stephen E. Richardson of the FBI’s Criminal Investigative Divisionmade the announcement. Jeffery Van Dyke, 46, was charged on April 4, 2016, and pleaded guilty before U.S. District Judge T.S. Ellis III of the Eastern District of Virginia. Sentencing is set for June 9. According to admissions made in connection with the plea agreement, members of the conspiracy created false profiles on social networking sites popular with children posing as young teenagers to lure children to two websites they controlled. Once on the conspirators’ websites, Van Dyke admitted that members of the conspiracy showed the children pre-recorded videos of prior minor victims, often engaging in sexually explicit conduct, to make the new victims think that they were chatting with another minor. Van Dyke further admitted that conspirators used these videos to coerce and entice children to engage in sexually explicit activity on their own web cameras, which could be viewed live by other members without the victim’s knowledge and which the website automatically recorded and made available for download later. Van Dyke admitted that he linked minors to one of the websites and chatted with them there in furtherance of the conspiracy. The defendant also admitted that one of the websites ranked the efforts of the members to successfully coerce and entice children to engage in sexually explicit conduct on live web camera. Law enforcement agencies have disabled both websites. VCACS special agents led the investigation with the assistance of the FBI’s Operation Rescue Me and the FBI’s Digital Analysis and Research Center and the Office of Victim Assistance. The South Africa Police Service, Family Violence, Child Protection and Sexual Offenses, Gauteng; Royal Canadian Mounted Police, National Child Exploitation Coordination Centre; the Dutch Police Service Agency, KLPD; and the Australian Federal Police, Child Protection Operations, Sydney were active partners in Operation Subterfuge, a multinational investigation coordinated by members of the FBI’s Violent Crimes Against Children International Task Force. Trial Attorney Lauren Britsch of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Whitney Russell of the Eastern District of Virginia prosecuted the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ Offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.","A Weed, California man was sentenced to 218 months in prison for conspiracy to produce child pornography based on his participation in a website that was operated for the purpose of coercing and enticing minors as young as eight years old to engage in sexually explicit conduct on web camera. Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division; U.S. Attorney Dana J. Boente of the Eastern District of Virginia; and Section Chief John J. Brosnan of the FBI’s Violent Crimes Against Children Section (VCACS) made the announcement. Jeffery Van Dyke, 46, was charged on April 4, 2016, and pleaded guilty before U.S. District 

**Do they seem like different stages of the same crime or just press releases covering similar crimes?** In both pairs, the same crime seems to be detaailed (1-"Church Hill, Maryland", 2- "Weed, California"). However, they seem to be press releases covering different stages of these respective cases. It seems that one was written to cover the plea, and the other was written to cover the sentence, saying how long the perpetrator would serve in prison.